<a href="https://colab.research.google.com/github/kevin-weis/Hit_Song_Science/blob/main/Hit_Prediction_Data_Collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install spotipy

     |████████████████████████████████| 61kB 4.7MB/s 
     |████████████████████████████████| 143kB 21.0MB/s 
ERROR: google-colab 1.0.0 has requirement requests~=2.23.0, but you'll have requests 2.25.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm import tqdm_notebook
import spotipy
import pandas as pd
import numpy as np

client_id = 'b0a721515fbe4aa689f4685942d43527'
client_secret = 'c788d362a41645fc9d30369195d66d65'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=100)

In [ ]:
def analyze_playlist(creator, playlist_id):

  #create empty dataframe
  playlist_features_list = ['artist', 'album', 'track_name', 'track_id', 
                            'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
                            'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']
  playlist_df = pd.DataFrame(columns = playlist_features_list)

  #create empty dict
  playlist_features = {}

  #loop through every track in the playlist, extract features, and append the features to the playlist df
  playlist = sp.user_playlist_tracks(creator, playlist_id)['items']
  for track in playlist:
    #get metadata
    playlist_features['artist'] = track['track']['album']['artists'][0]['name']
    playlist_features['album'] = track['track']['album']['name']
    playlist_features['track_name'] = track['track']['name']
    playlist_features['track_id'] = track['track']['id']

    #get audio features
    audio_features = sp.audio_features(playlist_features['track_id'])[0]
    for feature in playlist_features_list[4:]:
      playlist_features[feature] = audio_features[feature]

    #concat the dfs
    track_df = pd.DataFrame(playlist_features, index = [0])
    playlist_df = pd.concat([playlist_df, track_df], ignore_index= True)
  
  return playlist_df

In [ ]:
# get all playlists created by spotify as starting point for creating dataset of songs and their audio features
spotify_playlists = sp.user_playlists(user='spotify')

In [ ]:
df = pd.DataFrame(columns=['artist', 'album', 'track_name', 'track_id', 
                           'danceability', 'energy', 'key', 'loudness',
                           'mode', 'speechiness', 'instrumentalness',
                           'liveness', 'valence', 'tempo', 'duration_ms',
                           'time_signature'])

In [ ]:
for playlists in range(0, len(spotify_playlists['items'])):
  pl_id = 'spotify:playlist:'+spotify_playlists['items'][playlists]['id']
  try:
    df = df.append(analyze_playlist("spotify:user:spotify", str(pl_id)))
  except:
    pass

In [ ]:
df.shape

(3072, 16)

In [ ]:
unique_artists = list(set(df['artist']))
print(unique_artists[:5])
print(len(unique_artists))

['Kamo Mphela', 'JID', 'Vedo', 'LPB Poody', 'Leonard Anderson']
1596


In [ ]:
#use related artists of each of these artists to get the ids of all the related artists
related_artists = []
search = []
for artist in unique_artists:
  ar = sp.search(artist)
  ar_id = ar['tracks']['items'][0]['artists'][0]['id']
  artist_id = "spotify:artist:"+ar_id
  related_artists.append(sp.artist_related_artists(artist_id))


In [ ]:
related_artists[1]['artists'][0]['id']

'5REHfa3YDopGOzrxwTsPvH'

In [ ]:
len(related_artists)

1596

In [ ]:
rl_ids = []
related_ids = []
for i in range(len(related_artists)):
  rl_dict = related_artists[i]
  try:
    for i in range(0, len(rl_dict['artists'])-1):
      rl_id = rl_dict['artists'][i]['id']
      rl_ids.append("spotify:artist:"+rl_id)
  except:
    pass

In [ ]:
print(rl_ids[:3])

['spotify:artist:1nVEvn7RMNxj27rn0WE13E', 'spotify:artist:0Mu3PMBPwtQh5iggGTmudK', 'spotify:artist:2Hj2zBCjnAnmLmNIyjuMGo']


In [ ]:
print(len(rl_ids))

29612


In [ ]:
unique_ids = list(set(rl_ids))
len(unique_ids)

9680

In [ ]:
# get song ids for every song from all these artists to create a large dataset of song ids that can be used for audio feature extraction
song_ids = []
for id in unique_ids:
  albums = sp.artist_albums(id)
  try:
    for i in range(0, len(albums)):
      album_id = albums['items'][i]['id']
      album_tracks = sp.album_tracks(album_id)
      for i in album_tracks.items():
        s=album_tracks['items']
        for i in range(len(s)-1):
          song_ids.append(s[i]['uri'])
  except:
    pass

In [ ]:
song_ids[:5]

['spotify:track:4Kh6cq3XgOGUK9yXQu7RTq',
 'spotify:track:64zyMkiyBPSyCUgLl4AIXB',
 'spotify:track:4Kh6cq3XgOGUK9yXQu7RTq',
 'spotify:track:64zyMkiyBPSyCUgLl4AIXB',
 'spotify:track:4Kh6cq3XgOGUK9yXQu7RTq']

In [ ]:
print(len(list(set(song_ids))))

554363


In [ ]:
unique_songs = list(set(song_ids))

NameError: ignored

In [ ]:
song_feature_list = ['danceability', 'energy', 'key',
                     'loudness', 'mode', 'speechiness',
                     'instrumentalness', 'liveness', 'valence',
                     'acousticness',
                     'tempo', 'duration_ms', 'time_signature']
songs_df = pd.DataFrame(columns=song_feature_list)
for song in song_ids:
  try:
    audio_features = sp.audio_features(song)
    af = audio_features[0]
    song_features = {}
    if af != None:
      try:
        song_features['id'] = af['uri']
        song_features['danceability'] = af['danceability']
        song_features['energy'] = af['energy']
        song_features['key'] = af['key']
        song_features['loudness'] = af['loudness']
        song_features['mode'] = af['mode']
        song_features['speechiness'] = af['speechiness']
        song_features['acousticness'] = af['acousticness']
        song_features['instrumentalness'] = af['instrumentalness']
        song_features['liveness'] = af['liveness']
        song_features['valence'] = af['valence']
        song_features['tempo'] = af['tempo']
        song_features['duration_ms'] = af['duration_ms']
        song_features['time_signature'] = af['time_signature']

        song_df = pd.DataFrame(song_features, index=[0])
        songs_df = pd.concat([songs_df, song_df], ignore_index=True)
      except:
        pass
    else:
      pass
  except:
    pass